In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/lstm.csv')
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [3]:
df.category.unique()

array(['food', 'sports'], dtype=object)

In [5]:
# 데이터에 사용된 중복되지 않는 전체 단어 수를 파악
results = set()
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size

536

In [6]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list()
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [7]:
from tensorflow import keras

In [8]:
keras.utils.set_random_seed(1)

In [9]:
# one hot encodin
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph, vocab_size) for paragraph in paragraphs]
encoded_paragraphs

[[364,
  230,
  144,
  187,
  301,
  348,
  169,
  230,
  134,
  52,
  143,
  212,
  278,
  82,
  449,
  484,
  500,
  24,
  176,
  191,
  394,
  120,
  117,
  321,
  489,
  304,
  220,
  401,
  24,
  328,
  301,
  108,
  454,
  106,
  256,
  115,
  334,
  37,
  395,
  384,
  300,
  220,
  354,
  324,
  216,
  228,
  309,
  68,
  251,
  143,
  487,
  24,
  248],
 [310,
  449,
  484,
  262,
  191,
  384,
  117,
  278,
  22,
  345,
  376,
  240,
  304,
  298,
  19,
  198,
  461,
  120,
  342],
 [447,
  129,
  462,
  76,
  216,
  515,
  304,
  527,
  180,
  480,
  129,
  395,
  33,
  527,
  216,
  515,
  230,
  155,
  95,
  34,
  7,
  510,
  54,
  155,
  214,
  461,
  301,
  24,
  208,
  190,
  68,
  129,
  88,
  304,
  392,
  121,
  24,
  248,
  95,
  132,
  108,
  256,
  328],
 [432,
  153,
  187,
  11,
  185,
  143,
  213,
  304,
  158,
  341,
  304,
  213,
  236,
  143,
  360,
  37,
  57,
  304,
  143,
  236,
  37,
  33,
  391,
  231,
  434,
  252,
  143,
  236,
  74,
  251,
  295,
  

In [10]:
# 데이터에서 가장 긴 문장의 단어 수 확인
max_length = 0
for row in df['paragraph']:
    if len(row.split(' ')) > max_length:
        max_length = len(row.split(' '))
print(max_length)

91


In [11]:
# 문장마다 단어 수가 다르므로 sequence padding을 넣어서 문장의 길이를 동일하게 만들기
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(encoded_paragraphs, maxlen=max_length, padding='post')
padded_paragraphs_encoding

array([[364, 230, 144, ...,   0,   0,   0],
       [310, 449, 484, ...,   0,   0,   0],
       [447, 129, 462, ...,   0,   0,   0],
       ...,
       [509, 291, 108, ...,   0,   0,   0],
       [278, 143, 453, ...,   0,   0,   0],
       [477, 401, 143, ...,   0,   0,   0]], dtype=int32)

> 'post'는 뒤에 0으로 채워진다.

In [12]:
# 분류항목 'food, sports'를 수치로 분류하기
categories = df['category'].to_list()

def category_encode(category):
    if category == 'food':
        return [1,0]
    else:
        return [0,1]

In [13]:
encoded_category = [category_encode(category) for category in categories]
encoded_category

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [14]:
# Feature 확인
print(encoded_paragraphs[0])

[364, 230, 144, 187, 301, 348, 169, 230, 134, 52, 143, 212, 278, 82, 449, 484, 500, 24, 176, 191, 394, 120, 117, 321, 489, 304, 220, 401, 24, 328, 301, 108, 454, 106, 256, 115, 334, 37, 395, 384, 300, 220, 354, 324, 216, 228, 309, 68, 251, 143, 487, 24, 248]


In [15]:
print(len(encoded_paragraphs[0]))

53


In [16]:
len(encoded_paragraphs[19])

73

---
### 주제를 분류하는 모델 구현하기

In [17]:
model = keras.Sequential()
# 문맥 생성 단계
model.add(keras.layers.Embedding(vocab_size, 5, input_length=max_length))
# embedding 레이어는 인덱스를 받아 5차원 벡터의 embedding을 출력합니다. 
# 5차원인 이유는 과거, 과거진행, 과거완료, 현재, 현재진행, 현재완료 ? 아님 5형식?
model.add(keras.layers.LSTM(64))
# 분류단계
model.add(keras.layers.Dense(32, activation='relu')) # LSTM 반값
model.add(keras.layers.Dense(2, activation='softmax')) # 문장은 확률값을 구해야하므로

2022-07-22 14:20:03.215403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [19]:
import numpy as np

In [20]:
# list to numpy
train_X = np.array(padded_paragraphs_encoding)
train_y = np.array(encoded_category)

In [21]:
model.fit(train_X, train_y, batch_size=10, epochs=50)

Epoch 1/50
2/2 [==============================] - 2s 20ms/step - loss: 0.6941 - accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6924 - accuracy: 0.5500
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6927 - accuracy: 0.5500
Epoch 5/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6924 - accuracy: 0.5000
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6920 - accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6916 - accuracy: 0.5000
Epoch 8/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6912 - accuracy: 0.6000
Epoch 9/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6907 - accuracy: 0.6000
Epoch 10/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6904 - accuracy: 0.6000
Epoch 11/50
2/2 [============